In [1]:
import codenamesLLM
import pandas as pd
import openpyxl
from tqdm import tqdm

c:\Users\miche\OneDrive\Desktop\Codenames Progetto PDE\Codenames-LLM\codenames_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Path to your Excel file
file_path = "experiment_data\model_tournament\model_tournament_mich1.xlsx" #compy the model_tournament_input to use one new

# Read the Excel file into a DataFrame
df = pd.read_excel(file_path)

# Iterate through the rows using titled columns
for index, row in tqdm(df.iterrows(), total=len(df)):
    # Access values using column titles
    red_team = row['red_model']  # Replace with the actual column title for the red team
    blue_team = row['blue_model']  # Replace with the actual column title for the blue team
    playable = True

    already_played = pd.notna(row['winner'])
    
    if ((pd.notna(red_team) and pd.notna(blue_team)) and not(already_played)):  # Check if both values are not NaN
        try:
            print(f"Playing {red_team} vs {blue_team}...")
            # Call your function and get the result
            result = codenamesLLM.play_game(red_model = red_team, blue_model = blue_team)
            print(result)
            red_stats = codenamesLLM.analyze_team_guesses(result[3], "RED")
            blue_stats = codenamesLLM.analyze_team_guesses(result[3], "BLUE")

            df.at[index, 'red_model'] = red_team
            df.at[index, 'blue_model'] = blue_team
            df.at[index, 'winner'] = result[0]  
            df.at[index, 'red_avg_words_2guess'] = red_stats['average_expected_guesses']
            df.at[index, 'blue_avg_words_2guess'] = blue_stats['average_expected_guesses']
            df.at[index, 'red_avg_words_guessed'] = red_stats['average_correct_guesses']
            df.at[index, 'blue_avg_words_guessed'] = blue_stats['average_correct_guesses']
            df.at[index, 'reason'] = result[1]
            df.at[index, 'red_turns'] = red_stats['total_hints']
            df.at[index, 'blue_turns'] = blue_stats['total_hints']
            df.at[index, 'red_cib'] = result[4]
            df.at[index, 'blue_cib'] = result[5]

        except Exception as e:
            print(f"skipped game between {red_team} and {blue_team}: {e}")

    # Write the updated DataFrame back to the same Excel file
    df.to_excel(file_path, index=False)
    if not already_played:
        print(f"Executed {red_team} vs {blue_team}")


100%|██████████| 90/90 [00:03<00:00, 23.20it/s]


In [ ]:
# Path to your Excel file
file_path = "experiment_data\cot_data\cot_data.xlsx" #compy the model_tournament_input to use one new

# Read the Excel file into a DataFrame
df = pd.read_excel(file_path)

# Iterate through the rows using titled columns
for index, row in tqdm(df.iterrows(), total=len(df)):
    # Access values using column titles
    red_cot = bool(row['red_cot'])  # Replace with the actual column title for the red team
    blue_cot = bool(row['blue_cot'])  # Replace with the actual column title for the blue team
    #print(red_cot, blue_cot)
    already_played = pd.notna(row['winner'])
    
    if ((pd.notna(red_cot) and pd.notna(blue_cot)) and not(already_played)):  # Check if both values are not NaN
        try:
            print(f"Playing {index+1}/{len(df)} game...")
            # Call your function and get the result
            result = codenamesLLM.play_game(red_model="gpt-4o-mini", blue_model="gpt-4o-mini", red_cot=red_cot, blue_cot=blue_cot)
            print(result)
            red_stats = codenamesLLM.analyze_team_guesses(result[3], "RED")
            blue_stats = codenamesLLM.analyze_team_guesses(result[3], "BLUE")

            df.at[index, 'red_cot'] = red_cot
            df.at[index, 'blue_cot'] = blue_cot
            df.at[index, 'winner'] = result[0]  
            df.at[index, 'red_avg_words_2guess'] = red_stats['average_expected_guesses']
            df.at[index, 'blue_avg_words_2guess'] = blue_stats['average_expected_guesses']
            df.at[index, 'red_avg_words_guessed'] = red_stats['average_correct_guesses']
            df.at[index, 'blue_avg_words_guessed'] = blue_stats['average_correct_guesses']
            df.at[index, 'reason'] = result[1]
            df.at[index, 'red_turns'] = red_stats['total_hints']
            df.at[index, 'blue_turns'] = blue_stats['total_hints']
            df.at[index, 'red_cib'] = result[4]
            df.at[index, 'blue_cib'] = result[5]

        except Exception as e:
            print(f"skipped game: {e}")

    # Write the updated DataFrame back to the same Excel file
    df.to_excel(file_path, index=False)
    if not already_played:
        print(f"Executed {red_team} vs {blue_team}")

  0%|          | 0/48 [00:00<?, ?it/s]

True False
Playing 0/48 game...
